Continuing with the previous example, it will be common to have more than one related model.

This is especially the case for user models, because:

- The `input model` needs to be able to have a password.
- The `output model` should not have a password.
- The `database model` would probably need to have a hashed password.

### Multiple models

In [3]:
from fastapi import FastAPI
from pydantic import BaseModel, EmailStr

app = FastAPI()

In [5]:
class UserIn(BaseModel): 
    username: str 
    password: str 
    email: EmailStr 
    full_name: str | None = None

class UserOut(BaseModel):
    username: str 
    email: EmailStr 
    full_name: str | None = None 

class UserInDB(BaseModel): 
    username: str 
    hashed_password: str 
    email: EmailStr 
    full_name: str | None = None

In [6]:
def fake_password_hasher(raw_password: str): 
    return "supersecret" + raw_password

In [9]:
def fake_save_user(user_in: UserIn): 
    hashed_password = fake_password_hasher(user_in.password) 
    # user_in_db = UserInDB(**user_in.dict(), hashed_password=hashed_password) 
    user_in_db = UserInDB(**user_in.model_dump(), hashed_password=hashed_password) 
    return user_in_db

In [10]:
@app.post("/user/", response_model=UserOut) 
async def create_user(user_in: UserIn): 
    user_saved = fake_save_user(user_in)
    return user_saved

#### Unpacking a `dict` 

`UserInDB(**user_dict)` is equivalent to: 

```python
UserInDB(
    username="john",
    password="secret",
    email="john.doe@example.com",
    full_name=None,
)
```

Or more exactly, using user_dict directly, with whatever contents it might have in the future:

```python 
UserInDB(
    username = user_dict["username"],
    password = user_dict["password"],
    email = user_dict["email"],
    full_name = user_dict["full_name"],
)
```

## Reduce duplication 

refactoring above to avoid duplicate code 

In [11]:
class UserBase(BaseModel): 
    username: str 
    email: EmailStr 
    full_name: str | None = None 

class UserIn(UserBase): 
    password: str 

class UserOut(UserBase): 
    pass 

class UserInDB(UserBase): 
    hashed_password: str 


In [12]:
def fake_password_hasher(raw_password: str):
    return "supersecret" + raw_password


def fake_save_user(user_in: UserIn):
    hashed_password = fake_password_hasher(user_in.password)
    user_in_db = UserInDB(**user_in.dict(), hashed_password=hashed_password)
    print("User saved! ..not really")
    return user_in_db


@app.post("/user/", response_model=UserOut)
async def create_user(user_in: UserIn):
    user_saved = fake_save_user(user_in)
    return user_saved

### `Union` or `anyOf`

In [13]:
from typing import Union

from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()


class BaseItem(BaseModel):
    description: str
    type: str


class CarItem(BaseItem):
    type: str = "car"


class PlaneItem(BaseItem):
    type: str = "plane"
    size: int


items = {
    "item1": {"description": "All my friends drive a low rider", "type": "car"},
    "item2": {
        "description": "Music is my aeroplane, it's my aeroplane",
        "type": "plane",
        "size": 5,
    },
}


@app.get("/items/{item_id}", response_model=Union[PlaneItem, CarItem])
async def read_item(item_id: str):
    return items[item_id]


In this example we pass Union[PlaneItem, CarItem] as the value of the argument response_model.

Because we are passing it as a value to an argument instead of putting it in a type annotation, we have to use Union even in Python 3.10.

If it was in a type annotation we could have used the vertical bar, as:


some_variable: PlaneItem | CarItem

But if we put that in the assignment response_model=PlaneItem | CarItem we would get an error, because Python would try to perform an invalid operation between PlaneItem and CarItem instead of interpreting that as a type annotation.



### List of models 


In [14]:

from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()


class Item(BaseModel):
    name: str
    description: str


items = [
    {"name": "Foo", "description": "There comes my hero"},
    {"name": "Red", "description": "It's my aeroplane"},
]


@app.get("/items/", response_model=list[Item])
async def read_items():
    return items

### Response with arbitrary `dict`

You can also declare a response using a plain arbitrary dict, declaring just the type of the keys and values, without using a Pydantic model.

This is useful if you don't know the valid field/attribute names (that would be needed for a Pydantic model) beforehand.

In [16]:
@app.get("/keyword-weights/", response_model=dict[str, float])
async def read_keyword_weights():
    return {"foo": 2.3, "bar": 3.4}